<a href="https://colab.research.google.com/github/sailubandi/Player-Re-Identification-in-a-Single-Feed/blob/main/Player-Re-Identification-in-a-Single-Feed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/computer\ vision-20250709T052053Z-1-001.zip

Archive:  /content/computer vision-20250709T052053Z-1-001.zip
  inflating: computer vision/broadcast.mp4  
  inflating: computer vision/tacticam.mp4  
  inflating: computer vision/best.pt  


In [2]:
!pip install ultralytics opencv-python-headless filterpy deep_sort_realtime --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━

In [3]:
from ultralytics import YOLO
import cv2
import os



# Load the YOLO model (change to correct path)
model = YOLO('/content/computer vision/best.pt')  # Update with your model path


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
from deep_sort_realtime.deepsort_tracker import DeepSort

# Deep SORT with tuned parameters for better re-ID
tracker = DeepSort(
    max_age=60,                # Keep ID for 60 frames if object disappears
    n_init=2,                  # Confirm object after 2 detections
    max_cosine_distance=0.35, # Better for similar-looking players
    nn_budget=100,            # Memory budget for appearance embeddings
    override_track_class=None # Track only 'player' class from detection
)


In [9]:
import cv2

video_path = "/content/15sec_input_720p.mp4"  # Input video path
cap = cv2.VideoCapture(video_path)

# Output video setup
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('/content/output_tracked.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 Detection
    results = model(frame, verbose=False)[0]
    detections = []

    for box in results.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Keep only class 0 (players), with sufficient confidence
        if cls == 0 and conf > 0.4:
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'player'))

    # Deep SORT Tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f"Player {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # Frame info for debugging
    cv2.putText(frame, f"Frame: {frame_id} | Detected: {len(detections)}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    frame_id += 1

    out.write(frame)

cap.release()
out.release()
print("✅ Done! Output saved at: /content/output_tracked.mp4")


✅ Done! Output saved at: /content/output_tracked.mp4
